In [1]:
import json
import glob
import os.path
import pandas as pd
import numpy as np


In [2]:
def gettblData(tbl):# hitter 정보 테이블
    li = []
    for row in tbl['rows']:
        subli = []
        for r in row['row']:
            subli.append(r['Text'])
        li.append(subli)
    return pd.DataFrame(li)

# 타자 기록 2팀.
def get_hitter_record(path):
    boxscoreData = json.loads(open(path, 'r', encoding='utf-8-sig').read())

    result = []
    for hitter in boxscoreData['arrHitter']:
        df_hitter_record = pd.DataFrame()
        # tbl1 # 이닝/포지션/이름
        # tbl2 # 이닝별 성적
        # tbl3 # 타수 안타 타점 득점 타율
        # hitter = boxscoreData['arrHitter'][1] # table1 ~ 3
        for key in hitter: # 3구역으로 구성된 hitter 테이블을 하나씩 pick
            tbl = json.loads(hitter[key])
            df_hitter_record = pd.concat([df_hitter_record, gettblData(tbl)], axis=1)

        # 컬럼 이름 지정
        idx = ['이닝','포지션','선수명']
        idx.extend([str(i+1) for i in list(range(boxscoreData['realMaxInning']))])
        idx.extend(['타수','안타','타점','득점','타율'])
        df_hitter_record.columns = idx
        result.append(df_hitter_record)
    return result

def get_Pitcher_record(path):
    boxscoreData = json.loads(open(path, 'r', encoding='utf-8-sig').read())
    result = []

    for pitcher in boxscoreData['arrPitcher']:
        tbl = json.loads(pitcher['table'])
        pitcher_df = gettblData(tbl)
        pitcher_df.columns = ['선수명','등판','결과','승','패','세','이닝','타자','투구수','타수','피안타','홈런','4사구','삼진','실점','자책','평균자책점']
        result.append(pitcher_df)
    return result

def get_Summary(path):
    boxscoreData = json.loads(open(path, 'r', encoding='utf-8-sig').read())
    data = json.loads(boxscoreData['tableEtc'])
    return gettblData(data)

def getScoreBoard(path):
    data = json.loads(open(path, 'r', encoding='utf-8-sig').read())

    tbl1_data = json.loads(data['table1'])
    tbl2_data = json.loads(data['table2'])
    tbl3_data = json.loads(data['table3'])

    scoreboardDF = {}
    scoreboardDF['away_승패'] = tbl1_data['rows'][0]['row'][0]['Text']
    scoreboardDF['away_전적'] = tbl1_data['rows'][0]['row'][1]['Text'].split('</span>')[1]
    inning = 1
    for i in tbl2_data['rows'][0]['row']:
        scoreboardDF['away_' + str(inning)] = i['Text']
        inning += 1

    scoreboardDF['away_R'] = tbl3_data['rows'][0]['row'][0]['Text'] # R 점수
    scoreboardDF['away_H'] = tbl3_data['rows'][0]['row'][1]['Text'] # H 안타
    scoreboardDF['away_E'] = tbl3_data['rows'][0]['row'][2]['Text'] # E 에러
    scoreboardDF['away_B'] = tbl3_data['rows'][0]['row'][3]['Text'] # 볼넷 수

    scoreboardDF['home_승패'] = tbl1_data['rows'][1]['row'][0]['Text']
    scoreboardDF['home_전적'] = tbl1_data['rows'][1]['row'][1]['Text'].split('</span>')[1]
    inning = 1
    for i in tbl2_data['rows'][1]['row']:
        scoreboardDF['home_' + str(inning)] = i['Text']
        inning += 1
    scoreboardDF['home_R'] = tbl3_data['rows'][1]['row'][0]['Text'] # R 점수
    scoreboardDF['home_H'] = tbl3_data['rows'][1]['row'][1]['Text'] # H 안타
    scoreboardDF['home_E'] = tbl3_data['rows'][1]['row'][2]['Text'] # E 에러
    scoreboardDF['home_B'] = tbl3_data['rows'][1]['row'][3]['Text'] # 볼넷 수



In [15]:
# SR_ID
# 포스트 시즌 : 4
# 시범경기 : 1
# 정규시즌 : 0


## test용 파일 경로
# filename1 = r'0_2021_20210403SSWO0_GetBoxScoreScroll.json'
# filename2 = r'0_2021_20210403SSWO0_GetScoreBoardScroll.json'
# boxscorePath = os.path.join(dirpath, filename1)
# scoreboardPath = os.path.join(dirpath, filename2)

# get_Gameinfo(scoreboardPath)
# getScoreboardDataFrame(scoreboardPath) # 스코어보드


## box score 정리
# get_Summary(boxscorePath) # 요약 정보
# get_hitter_record(boxscorePath)[0] # 타자 기록 : 원정팀
# get_hitter_record(boxscorePath)[1] # 타자 기록 : 홈팀
# get_Pitcher_record(boxscorePath)[0] # 투수 기록 : 원정팀
# get_Pitcher_record(boxscorePath)[1] # 투수 기록 : 홈팀

,0
SR_ID,0
G_ID,20210403SSWO0
G_DT,2021-04-03
SEASON_ID,2021
HOME_NM,키움
HOME_ID,WO
AWAY_NM,삼성
AWAY_ID,SS
S_NM,고척
CROWD_CN,"1,665"


In [3]:

result = pd.DataFrame()

dirpath = r'..\..\..\DataSet\KBO\play_ScoreBoard_RawData_json'
for filename in os.listdir(dirpath):
    if filename.__contains__('GetScoreBoardScroll') == False:
        continue
    path = os.path.join(dirpath,filename)
    result = result.append(getScoreBoard(path))
    # print(f'>>> {path} is Loading Complete!!!')

print(f'>>> all files is Loading Complete!!!')

result.reset_index(drop=True, inplace=True)
# result.to_csv('playReview_ScoreBoard_Rawdata.csv', encoding='utf-8-sig', index=False)



# result.loc[result['SEASON_ID'] == 2018].to_csv('playReview_ScoreBoard_Rawdata_2018.csv', encoding='utf-8-sig', index=False)
# result.loc[result['SEASON_ID'] == 2019].to_csv('playReview_ScoreBoard_Rawdata_2019.csv', encoding='utf-8-sig', index=False)
# result.loc[result['SEASON_ID'] == 2020].to_csv('playReview_ScoreBoard_Rawdata_2020.csv', encoding='utf-8-sig', index=False)
# result.loc[result['SEASON_ID'] == 2021].to_csv('playReview_ScoreBoard_Rawdata_2021.csv', encoding='utf-8-sig', index=False)

# 2018 year :: 720 rows
# 2019 year :: 720 rows
# 2020 year :: 720 rows
# 2021 year :: 721 rows

# return pd.concat([pd.DataFrame(data, index=[0]), pd.DataFrame(scoreboardDF, index=[0])], axis=1)

>>> all files is Loading Complete!!!


In [63]:
dirPath = r'../../../DataSet/KBO/play_ScoreBoard_RawData_json/'

scoreboard_BaseInfo = pd.DataFrame()
scoreboard_Table1 = pd.DataFrame()
scoreboard_Table2_HOME = pd.DataFrame()
scoreboard_Table2_AWAY = pd.DataFrame()
scoreboard_Table3_HOME = pd.DataFrame()
scoreboard_Table3_AWAY = pd.DataFrame()


for fileName in sorted(os.listdir(dirPath)):
    jsobj = json.loads(open(
        os.path.join(dirPath,fileName),
        'r', encoding='utf-8-sig').read())

    ## Baseinfo Parsing
    scoreboard_BaseInfo = scoreboard_BaseInfo.append(pd.json_normalize(jsobj))
    del scoreboard_BaseInfo['table1']
    del scoreboard_BaseInfo['table2']
    del scoreboard_BaseInfo['table3']


    ## table 1 Parsing
    tmp = pd.json_normalize(json.loads(jsobj['table1']), record_path=['rows','row'])['Text']
    tmp[1] = tmp[1].split('</span>')[1]
    tmp[3] = tmp[3].split('</span>')[1]
    tmp['G_ID'] = jsobj['G_ID']
    scoreboard_Table1 = scoreboard_Table1.append(tmp)


    ## table 2 Parsing
    tmp = pd.json_normalize(json.loads(jsobj['table2']), record_path=['rows','row'])['Text']
    tmp = pd.DataFrame(data=[tmp[:12],tmp[12:]])
    tmp.iloc[0] = tmp.iloc[0].shift(1)
    tmp.iloc[1] = tmp.iloc[1].shift(-11)
    tmp[0] = jsobj['G_ID']
    tmp = tmp.loc[:,:12]
    scoreboard_Table2_AWAY = scoreboard_Table2_AWAY.append(tmp.iloc[0])
    scoreboard_Table2_HOME = scoreboard_Table2_HOME.append(tmp.iloc[1])

    ## table 3 Parsing
    tmp = pd.json_normalize(json.loads(jsobj['table3']), record_path=['rows','row'])['Text']
    tmp = pd.DataFrame(data=[tmp[:4],tmp[4:]])
    tmp.iloc[0] = tmp.iloc[0].shift(1)
    tmp.iloc[1] = tmp.iloc[1].shift(-3)
    tmp = tmp.loc[:,:4]
    tmp[0] = jsobj['G_ID']
    scoreboard_Table3_AWAY = scoreboard_Table3_AWAY.append(tmp.iloc[0])
    scoreboard_Table3_HOME = scoreboard_Table3_HOME.append(tmp.iloc[1])

scoreboard_Table1.rename(columns={
    0:'AWAY_Score',
    1:'AWAY_GameRecord',
    2:'HOME_Score',
    3:'HOME_GameRecord'},
    inplace=True)

scoreboard_Table1['AWAY_Win'] = scoreboard_Table1['AWAY_GameRecord'].apply(lambda x: x.split(' ')[0])
scoreboard_Table1['AWAY_Lose'] = scoreboard_Table1['AWAY_GameRecord'].apply(lambda x: x.split(' ')[1])
scoreboard_Table1['AWAY_Draw'] = scoreboard_Table1['AWAY_GameRecord'].apply(lambda x: x.split(' ')[2])
scoreboard_Table1['HOME_Win'] = scoreboard_Table1['HOME_GameRecord'].apply(lambda x: x.split(' ')[0])
scoreboard_Table1['HOME_Lose'] = scoreboard_Table1['HOME_GameRecord'].apply(lambda x: x.split(' ')[1])
scoreboard_Table1['HOME_Draw'] = scoreboard_Table1['HOME_GameRecord'].apply(lambda x: x.split(' ')[2])

scoreboard_Table2_AWAY.rename(
    columns={
        0:'G_ID',
        1:'inning_01',
        2:'inning_02',
        3:'inning_03',
        4:'inning_04',
        5:'inning_05',
        6:'inning_06',
        7:'inning_07',
        8:'inning_08',
        9:'inning_09',
        10:'inning_10',
        11:'inning_11',
        12:'inning_12'},
    inplace=True)

scoreboard_Table2_HOME.rename(
    columns={
        0:'G_ID',
        1:'inning_01',
        2:'inning_02',
        3:'inning_03',
        4:'inning_04',
        5:'inning_05',
        6:'inning_06',
        7:'inning_07',
        8:'inning_08',
        9:'inning_09',
        10:'inning_10',
        11:'inning_11',
        12:'inning_12'},
    inplace=True)


scoreboard_Table3_AWAY.rename(
    columns={
        0:'G_ID',
        1:'R',
        2:'H',
        3:'E',
        4:'B'},
    inplace=True)

scoreboard_Table3_HOME.rename(
    columns={
        0:'G_ID',
        1:'R',
        2:'H',
        3:'E',
        4:'B'},
    inplace=True)

scoreboard_Table1['AWAY_Win'] = scoreboard_Table1['AWAY_Win'].apply(lambda x: str(x).replace('승',''))
scoreboard_Table1['AWAY_Lose'] = scoreboard_Table1['AWAY_Win'].apply(lambda x: str(x).replace('승',''))
scoreboard_Table1['AWAY_Draw'] = scoreboard_Table1['AWAY_Win'].apply(lambda x: str(x).replace('승',''))

scoreboard_Table1['HOME_Win'] = scoreboard_Table1['AWAY_Win'].apply(lambda x: str(x).replace('승',''))
scoreboard_Table1['HOME_Lose'] = scoreboard_Table1['AWAY_Win'].apply(lambda x: str(x).replace('승',''))
scoreboard_Table1['HOME_Draw'] = scoreboard_Table1['AWAY_Win'].apply(lambda x: str(x).replace('승',''))

In [71]:

# scoreboard_BaseInfo.tail()
scoreboard_BaseInfo.to_csv('scoreboard_BaseInfo_v1.csv', encoding='utf-8-sig', index=False)